In [1]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import sys
sys.path.append('/home/jovyan/ChestXray-14')

In [3]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, TensorBoard

2023-03-29 15:48:30.027040: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
ROOT_PATH = "/home/jovyan/ChestXray-14"
INPUT_PATH = f"{ROOT_PATH}/dataset/ChestXray NIH"

LABELS = ['No Finding']

weight_option = 'imagenet' # None or 'imagenet'
EXPERIMENT_NAME = "binary_classification_with_cross_entropy_loss"

tf.keras.mixed_precision
---
make model train faster
```python
tf.keras.mixed_precision.global_policy()
tf.keras.mixed_precision.set_global_policy('mixed_float16')
```

In [5]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA A100-SXM4-40GB MIG 2g.10gb, compute capability 8.0


## Model Training

In [6]:
import os
CURRENT_PATH = os.path.abspath("")
CURRENT_PATH

'/home/jovyan/ChestXray-14/experiments/Binary-Classification-Model'

In [7]:
# Learning rate
def lr_schedule(epoch, learning_rate):
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

In [8]:
import datetime

def get_callbacks(NAME, weight_option, fold_num=None):
    log_dir = f"{CURRENT_PATH}/logs/{EXPERIMENT_NAME}/{NAME}_{weight_option}_FOLD_{fold_num}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    model_checkpoint_callback = ModelCheckpoint(f'results/models/{EXPERIMENT_NAME}/{NAME}_{weight_option}_FOLD_{fold_num}.h5', monitor='val_loss', mode='min', save_best_only=True)
    early_stop_callback = EarlyStopping(monitor='val_loss', mode="min", patience=20, verbose=1)
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', mode="min", factor=0.5, patience=3, verbose=1)
    lr_logging_callback = LearningRateScheduler(lr_schedule)
    
    return model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback, tensorboard_callback

In [9]:
# Constant variables
NAME = "EfficientNetB0"
EPOCHS = 100
weight_option = None # use `'imagenet'` or `None` only

In [10]:
import time
from modules.models import ModelBinaryClassification
from modules.dataset import Dataset

NUM_FOLDS = 5
dataset = Dataset()
for fold_num in range(1, NUM_FOLDS + 1):
    
    # Callbacks
    (
        model_checkpoint_callback, 
        early_stop_callback, 
        reduce_lr_callback, 
        lr_logging_callback, 
        tensorboard_callback
    ) = get_callbacks(
        NAME, weight_option, fold_num
    )

    # Path for CSV
    path = os.path.join(CURRENT_PATH, "results", "history", EXPERIMENT_NAME, f"{NAME}_{weight_option}_FOLD_{fold_num}")
    os.makedirs(path, exist_ok=True)

    # CSV Logger
    csv_logger = CSVLogger(os.path.join(path, f"history.csv"))

    # Dataset
    train_dataset, test_dataset = dataset.get_kfold(fold_num)

    # Modeling
    transfer_model = tf.keras.applications.efficientnet.EfficientNetB0(
        include_top=False, 
        weights=weight_option,
        input_shape=(224, 224, 3),
        pooling=None
    )

    model = ModelBinaryClassification(
        transfer_model,
    )
    model = model.get_model()
    model._name = f"{EXPERIMENT_NAME}_FOLD_{fold_num}"
    model.summary()

    # Record time for training
    start = time.time()
    
    # Visualize
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=test_dataset,
        verbose=1, # Show Progress Bar while Traning
        callbacks=[model_checkpoint_callback, csv_logger, early_stop_callback, reduce_lr_callback, lr_logging_callback, tensorboard_callback]
    )
    
    end = time.time()
    print(
        "===== " * 5,
        "Model Traninig for: {:.2f} second(s)".format(end - start),
        "===== " * 5,
        "\n\n"
    )

2023-03-29 15:48:33.346821: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 15:48:34.114490: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8011 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 2g.10gb, pci bus id: 0000:31:00.0, compute capability: 8.0


Model: "binary_classification_with_cross_entropy_loss_FOLD_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 128)               8028288   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dens

2023-03-29 15:48:49.128850: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-03-29 15:48:49.837060: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-29 15:48:49.837882: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-29 15:48:49.837903: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-03-29 15:48:49.838643: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-29 15:48:49.838699: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


5606/5606 [==============================] - 627s 109ms/step - loss: 0.6859 - accuracy: 8.9190e-05 - auc: 0.5906 - f1_score: 0.6991 - val_loss: 0.6624 - val_accuracy: 0.0000e+00 - val_auc: 0.6370 - val_f1_score: 0.7032 - lr: 0.0010
Epoch 2/100
5606/5606 [==============================] - 602s 107ms/step - loss: 0.6528 - accuracy: 0.0000e+00 - auc: 0.6595 - f1_score: 0.6991 - val_loss: 0.6340 - val_accuracy: 0.0000e+00 - val_auc: 0.6979 - val_f1_score: 0.7032 - lr: 0.0010
Epoch 3/100
5606/5606 [==============================] - 599s 107ms/step - loss: 0.6279 - accuracy: 0.0000e+00 - auc: 0.7017 - f1_score: 0.6991 - val_loss: 0.6178 - val_accuracy: 0.0000e+00 - val_auc: 0.7148 - val_f1_score: 0.7032 - lr: 0.0010
Epoch 4/100
5606/5606 [==============================] - 602s 107ms/step - loss: 0.6155 - accuracy: 0.0000e+00 - auc: 0.7194 - f1_score: 0.6991 - val_loss: 0.6063 - val_accuracy: 0.0000e+00 - val_auc: 0.7330 - val_f1_score: 0.7032 - lr: 0.0010
Epoch 5/100
5606/5606 [=============

In [11]:
test_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))>